## ENUNCIADO
En el siguiente puzzle infantil se pide rellenar los espacios en blanco
con números del 1 al 9 de tal modo que la ecuación sea correcta siguiendo el
orden de las operaciones. Los números no pueden repetirse. Es decir, en cada
uno de los 9 huecos en blancos deberá ir un número sin repetirse. Según
declaraciones del profesor Tran Phuong existen 362.880 combinaciones
posibles para colocar los 9 dígitos en los 9 espacios en blanco, pero sólo unas
pocas son correctas

## ENTRADA
La entrada que recibe el problema es un array que representa la ecuacion:
    1. Los huecos se representan mediante "X"
    2. Ejemplo ["X","+",5,"=",7]
    3. Los numeros caen siempre en las posiciones pares, mientras que los operadores en las posiciones impares

## RESOLUCION
El estado va a ser una tupla, de tamaño 9+1+1=11:
    1. state[0]-state[8]: Las 9 primeras posiciones reflejan si el número ya ha sido usado (1) o no (0)
    2. state[9]: Se refiere a la posición de la ecuación en la que nos encontramos actualmente.
    3. state[10]: La última posición lleva el valor total hasta la posición state[9] - 2 en la ecuación.

#### OBSERVACIONES
    1.state[9] siempre va a ser par, pues solo puede referirse a números, y no a operadores.
    2.Al asignar un número a un hueco, debemos operar: (valor hasta hueco) operator (número asignado en hueco). Dicho 
    operator se encuentra en la posición state[9] - 1. De inicio el operator debe ser "+", pues el primer número seria equivalente a 0 + número. Por ello, a la ecuación dada le añadimos al principio [0,"+"]. Además, esto permite tener un único tipo de acción, en vez de tener que divir en dos subacciones.

#### ACCIONES
La acción consiste en avanzar hasta próximo hueco y asignar número en dicho hueco. Como el primer elemento de nuestra ecuación nunca va a ser un hueco,(artificialmente hemos añadido [0,"+"..]), siempre vamos a poder llevar a acabo la acción hasta que lleguemos al final de la ecuación.
Si el último hueco no se corresponde con el último operando en la ecuación, significa que debmos seguir operando hasta llegar 
al final sin asignar nada. Dicha acción se representa mediante el "0".
Por lo tanto, el factor de ramificación es 9, aunque según vayamos colocando números pasamos a tener k hijos siendo k=9-númerosColocados 

In [1]:
cd ..

C:\Users\aleja\Google Drive\4o\INFORMATICA\INTELIGENCIA ARTIFICIAL\PRACTICAS\PR1


In [2]:
cd ..

C:\Users\aleja\Google Drive\4o\INFORMATICA\INTELIGENCIA ARTIFICIAL\PRACTICAS


In [3]:
cd aima-python

C:\Users\aleja\Google Drive\4o\INFORMATICA\INTELIGENCIA ARTIFICIAL\PRACTICAS\aima-python


In [13]:
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [33]:
class Puzzle_Numeros (Problem):
    
    def __init__(self,ec):
        self.initial = (0,0,0,0,0,0,0,0,0,2,0)
        self.ecuation=[0,"+"]+ec
        self.goal = (1,1,1,1,1,1,1,1,1,len(self.ecuation)-1,self.ecuation[len(self.ecuation)-1])
    
    def actions(self, state):
        if (state[9]<(len(self.ecuation)-1) and state[9]%2==0): #Quedan elementos por comprobar, y estamos en un número
            l=[n for n in range (1,10) if state[n-1]==0]
            if l==[]: #Quedan números por operar en la ecuación, pero ya no tenemos más huecos
                l.append(0)
            return l
        else: return []

    def result(self, state, action):
        n=action
        l=list(state)
        if n>0:
            l[n-1]=1 #Marcamos como ya usado el número indicado por la acción
        i=l[9]
        stop=False
        #Primero avanzamos hasta el próximo hueco.
        #Después asignamos el número n al hueco correspondiente. Si n=0 nos saltamos este paso, pues hemos llegado al final.
        while (i<(len(self.ecuation)-1) and (not stop)):       
            if self.ecuation[i]=="X":
                x=n
                stop=True
            else:
                x=self.ecuation[i]
            if self.ecuation[i-1]=="+":
                l[10]+=x
            elif self.ecuation[i-1]=="-":
                l[10]-=x
            elif self.ecuation[i-1]=="*":
                l[10]=l[10]*x
            elif self.ecuation[i-1]==":":
                l[10]=l[10]/x
             #Nunca se va a meter en el igual, pues en ese caso i=len(self.ecuacion)-1, por lo que nos saldríamos del bucle
            i+=2
        l[9]=i     
        return tuple(l)
    
    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

In [34]:
p=Puzzle_Numeros(["X","+","X","*",5,":","X","+","X","+",12,"*","X","-","X","-",11,"*","X","*","X",":","X","-",10,"=",66])
p1=Puzzle_Numeros(["X","+","X","+","X","+","X","+","X","+","X","+","X","+","X","+","X","+",1,"-",1,"=",45])
p2=Puzzle_Numeros(["X","-","X","=",1])

In [31]:
depth_first_graph_search(p).solution()

[9, 5, 8, 7, 4, 6, 2, 1, 3, 0]

In [35]:
%%timeit
breadth_first_graph_search(p).solution()

5.12 s ± 20.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
uniform_cost_search(p).solution()

 A pesar de que nuestro árbol requiere más de 9! estados por la acción adicional de "avanzar", el cálculo del goal_test es constante; mientras que si solo llevamos los numeros situados, en una ecuacion de longitud n desconocida, el cálculo del goal_test sería lineal, lo que a la larga puede ser peor.